# UCR crime report api

This explores the [FBI Crime Data Explorer](https://crime-data-explorer.fr.cloud.gov/api), which allows API access to crime statistics. It is a new platform, and has quirks.

This may help to get [FIPS codes](http://nws.noaa.gov/mirs/public/prods/maps/map_images/state-maps/cnty_fips/tx_cnty.pdf).



In [1]:
import agate
import warnings
warnings.filterwarnings('ignore')

## codes

In [50]:
%%bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/codes?api_key=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv' > ../data/fbi-codes.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2336  100  2336    0     0   6066      0 --:--:-- --:--:-- --:--:--  6131


In [52]:
codes = agate.Table.from_json('../data/fbi-codes.json')

TypeError: When converting a JSON document with a top-level dictionary element, a key must be specified.

hate-test.json was downloaded using this
``` bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/hc/count/states/tx/bias_name?page=1&per_page=10&output=json&api_key=d769Gtaor7iHVrrcBHcQw8Jl7AWNqBYBzGwkLLkS' > fbi-hate.json
```

In [2]:
%%bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/hc/count/states/tx/bias_name?page=1&per_page=10&output=json&api_key=d769Gtaor7iHVrrcBHcQw8Jl7AWNqBYBzGwkLLkS' > ../data/fbi-hate.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 64587  100 64587    0     0  46315      0  0:00:01  0:00:01 --:--:-- 66515


In [3]:
# import
specified_types = {
    'year': agate.Text(),
}
hate = agate.Table.from_json(
    '../data/fbi-hate.json', column_types=specified_types, key='results'
)

In [4]:
print(hate)

| column    | data_type |
| --------- | --------- |
| year      | Text      |
| bias_name | Text      |
| count     | Number    |



In [5]:
hate.print_table()

| year | bias_name            | count |
| ---- | -------------------- | ----- |
| 1991 | Anti-Arab            |     1 |
| 1991 | Anti-Asian           |     6 |
| 1991 | Anti-Atheist/Agno... |     1 |
| 1991 | Anti-Black or Afr... |    26 |
| 1991 | Anti-Female Homos... |     2 |
| 1991 | Anti-Hispanic or ... |     7 |
| 1991 | Anti-Jewish          |    27 |
| 1991 | Anti-Male Homosex... |     9 |
| 1991 | Anti-Not Hispanic... |     2 |
| 1991 | Anti-Other Religion  |     1 |
| 1991 | Anti-White           |    14 |
| 1992 | Anti-American Ind... |     1 |
| 1992 | Anti-Arab            |    12 |
| 1992 | Anti-Asian           |    14 |
| 1992 | Anti-Black or Afr... |   201 |
| 1992 | Anti-Female Homos... |     4 |
| 1992 | Anti-Hispanic or ... |    23 |
| 1992 | Anti-Islamic (Mus... |     2 |
| 1992 | Anti-Jewish          |    34 |
| 1992 | Anti-Lesbian, Gay... |    10 |
| ...  | ...                  |   ... |


In [6]:
hate_year = hate.denormalize('bias_name', 'year', 'count')
print(hate_year)
hate_year.print_table()

| column    | data_type |
| --------- | --------- |
| bias_name | Text      |
| 1991      | Number    |
| 1992      | Number    |
| 1993      | Number    |
| 1994      | Number    |
| 1995      | Number    |
| 1996      | Number    |
| 1997      | Number    |
| 1998      | Number    |
| 1999      | Number    |
| 2000      | Number    |
| 2001      | Number    |
| 2002      | Number    |
| 2003      | Number    |
| 2004      | Number    |
| 2005      | Number    |
| 2006      | Number    |
| 2007      | Number    |
| 2008      | Number    |
| 2009      | Number    |
| 2010      | Number    |
| 2011      | Number    |
| 2012      | Number    |
| 2013      | Number    |
| 2014      | Number    |
| 2015      | Number    |
| 2016      | Boolean   |

| bias_name            | 1991 | 1992 | 1993 | 1994 | 1995 | ... |
| -------------------- | ---- | ---- | ---- | ---- | ---- | --- |
| Anti-Arab            |    1 |   12 |    4 |    5 |    4 | ... |
| Anti-Asian           |    6 |   14 |    9 |  

In [7]:
hate_sums = hate.pivot('bias_name', aggregation=agate.Sum('count'))
print(hate_sums)
hate_sums.print_table(max_column_width=40, max_rows=None)

| column    | data_type |
| --------- | --------- |
| bias_name | Text      |
| Sum       | Number    |

| bias_name                                |   Sum |
| ---------------------------------------- | ----- |
| Anti-Arab                                |   117 |
| Anti-Asian                               |   167 |
| Anti-Atheist/Agnosticism                 |     1 |
| Anti-Black or African American           | 2,668 |
| Anti-Female Homosexual (Lesbian)         |   167 |
| Anti-Hispanic or Latino                  |   642 |
| Anti-Jewish                              |   383 |
| Anti-Male Homosexual (Gay)               |   878 |
| Anti-Not Hispanic or Latino              |   253 |
| Anti-Other Religion                      |    64 |
| Anti-White                               |   802 |
| Anti-American Indian or Alaska Native    |    14 |
| Anti-Islamic (Muslem)                    |   131 |
| Anti-Lesbian, Gay, Bisexual, or Trans... |   182 |
| Anti-Multi-Racial Group                  |   

## agencies

list pulled from year 2010 `/participation/states/{state_abbr}`:

```bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/participation/agencies?page=1&per_page=2000&output=json&year=2010&state_name=Texas&api_key=d769Gtaor7iHVrrcBHcQw8Jl7AWNqBYBzGwkLLkS' > fbi-agencies.json
```

In [8]:
%%bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/participation/agencies?page=1&per_page=2000&output=json&year=2010&state_name=Texas&api_key=d769Gtaor7iHVrrcBHcQw8Jl7AWNqBYBzGwkLLkS' > ../data/fbi-agencies.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  400k  100  400k    0     0   259k      0  0:00:01  0:00:01 --:--:--  260k


In [9]:
agencies_types = {
    'year': agate.Text(),
}
agencies = agate.Table.from_json('../data/fbi-agencies.json', column_types=agencies_types, key='results')

In [10]:
print('number of records: {}'.format(len(agencies)))
print(agencies)
agencies.print_table()

number of records: 1113
| column                | data_type |
| --------------------- | --------- |
| year                  | Text      |
| state_name            | Text      |
| state_abbr            | Text      |
| agency_ori            | Text      |
| agency_name           | Text      |
| agency_population     | Number    |
| population_group_code | Text      |
| population_group      | Text      |
| reported              | Boolean   |
| months_reported       | Number    |
| nibrs_reported        | Boolean   |
| nibrs_months_reported | Number    |
| covered               | Boolean   |
| participated          | Boolean   |
| nibrs_participated    | Boolean   |

| year | state_name | state_abbr | agency_ori | agency_name          | agency_population | ... |
| ---- | ---------- | ---------- | ---------- | -------------------- | ----------------- | --- |
| 2010 | Texas      | TX         | TX0950100  | Abernathy            |             2,805 | ... |
| 2010 | Texas      | TX         | TX2

In [11]:
participated = agencies.where(lambda row: row['participated'] is True)
print(len(agencies))
print(len(participated))

1113
1077


In [12]:
participated.columns['agency_name'].values_distinct()

('Lone Oak',
 'Medina',
 'Independent School District: Austin',
 'Trophy Club',
 'Pecos',
 'Scurry',
 'Wichita Falls',
 'Garza',
 'Freestone',
 'Electra',
 'Denton',
 'Big Lake',
 'Anson',
 'Missouri City',
 'Frisco',
 'University of North Texas: Denton',
 'Irving',
 'Morton',
 'Randall',
 'Texarkana',
 'Fairfield',
 'Bovina',
 'Western Texas College',
 'Hillsboro',
 'Gatesville',
 'Northcrest',
 'La Grange',
 'Waco',
 'Kerr',
 'Mitchell',
 'Richardson',
 'University of Texas: Austin',
 'Tom Green',
 'Winkler',
 'Shackelford',
 'New Deal',
 'Sabinal',
 'Karnes City',
 'Ponder',
 'Nash',
 'Granite Shoals',
 'Rockwall',
 'Independent School District: Taft',
 'Penitas',
 'University of Texas: Tyler',
 'Ennis',
 'Seymour',
 'Wellington',
 'Alamo Heights',
 'Cibolo',
 'Anthony',
 'Burkburnett',
 'Fredericksburg',
 'Jones',
 'Pelican Bay',
 'Texas Medical Center',
 'University of North Texas: Health Science Center',
 'Collinsville',
 'Armstrong',
 'Wichita',
 'Shenandoah',
 'Early',
 "O'Donn

In [13]:
print(alist)

NameError: name 'alist' is not defined

## Suboffenses

In [53]:
%%bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/agencies/count/states/suboffenses/TX?page=1&per_page=100&output=json&api_key=d769Gtaor7iHVrrcBHcQw8Jl7AWNqBYBzGwkLLkS' > ../data/fbi-suboffense-tx.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39598  100 39598    0     0   1392      0  0:00:28  0:00:28 --:--:-- 11085


In [54]:
suboffense_types = {
    'year': agate.Text(),
    'agency_id': agate.Text()
}
suboffense = agate.Table.from_json(
    '../data/fbi-suboffense-tx.json',
    column_types=suboffense_types,
    key='results')
print('number of records: {}'.format(len(suboffense)))
print(suboffense)
suboffense.print_table()

number of records: 100
| column              | data_type |
| ------------------- | --------- |
| year                | Text      |
| agency_id           | Text      |
| offense_subcat_code | Text      |
| offense_name        | Text      |
| cleared             | Number    |
| offense_subcat_name | Text      |
| id                  | Number    |
| actual              | Number    |
| offense_id          | Number    |
| pub_agency_name     | Text      |
| state_postal_abbr   | Text      |
| unfounded           | Number    |
| juvenile_cleared    | Number    |
| offense_code        | Text      |
| ori                 | Text      |
| reported            | Number    |

| year | agency_id | offense_subcat_code | offense_name         | cleared | offense_subcat_name  | ... |
| ---- | --------- | ------------------- | -------------------- | ------- | -------------------- | --- |
| 2016 | 26310     | SUM_ASS_CUT         | Assault              |       0 | Assault - Knife o... | ... |
| 2016 | 2631

In [55]:
suboffense.select(['offense_name', 'pub_agency_name']).print_table(max_column_width=None)

| offense_name                     | pub_agency_name                    |
| -------------------------------- | ---------------------------------- |
| Assault                          | IndependentSchoolDistrict:Malakoff |
| Assault                          | IndependentSchoolDistrict:Malakoff |
| Assault                          | IndependentSchoolDistrict:Malakoff |
| Assault                          | IndependentSchoolDistrict:Malakoff |
| Burglary                         | IndependentSchoolDistrict:Malakoff |
| Burglary                         | IndependentSchoolDistrict:Malakoff |
| Burglary                         | IndependentSchoolDistrict:Malakoff |
| Burglary                         | IndependentSchoolDistrict:Malakoff |
| Larceny                          | IndependentSchoolDistrict:Malakoff |
| Motor Vehicle Theft              | IndependentSchoolDistrict:Malakoff |
| Motor Vehicle Theft              | IndependentSchoolDistrict:Malakoff |
| Motor Vehicle Theft              | I

## Specific agency

```bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/agencies/count/TX2210100/offenses?page=1&per_page=10&output=json&api_key=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv'
```


In [59]:
%%bash
curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/agencies/count/TX2210100/offenses?page=1&per_page=100&output=json&api_key=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv' > ../data/fbi-abeline.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27637  100 27637    0     0   3795      0  0:00:07  0:00:07 --:--:--  7221


In [56]:
# %%bash
# curl -X GET --header 'Accept: application/json' 'https://api.usa.gov/crime/fbi/ucr/agencies/count/TX2210100/offenses?page=1&per_page=1000&output=json&year>=2010&year&api_key=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv' > ../data/fbi-abeline.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0  26374      0  0:00:06  0:00:06 --:--:-- 41343


In [63]:
abeline_types = {
    'year': agate.Text(),
    'id': agate.Text(),
    'agency_id': agate.Text()
}

abeline = agate.Table.from_json(
    '../data/fbi-abeline.json',
    column_types=abeline_types,
    key='results')

# reseting table to include only years 2010 or greater
abeline = abeline.where(lambda row: row['year'] >= '2010')

print('number of records: {}'.format(len(abeline)))
print(abeline)
abeline.print_table()

number of records: 85
| column            | data_type |
| ----------------- | --------- |
| year              | Text      |
| offense_name      | Text      |
| offense_code      | Text      |
| state_postal_abbr | Text      |
| pub_agency_name   | Text      |
| id                | Text      |
| actual            | Number    |
| cleared           | Number    |
| unfounded         | Number    |
| juvenile_cleared  | Number    |
| agency_id         | Text      |
| ori               | Text      |
| reported          | Number    |

| year | offense_name         | offense_code | state_postal_abbr | pub_agency_name      | id      | ... |
| ---- | -------------------- | ------------ | ----------------- | -------------------- | ------- | --- |
| 2016 | Robbery              | SUM_ROB      | TX                | Abilene Police De... | 7026609 | ... |
| 2016 | Assault              | SUM_AST      | TX                | Abilene Police De... | 7126306 | ... |
| 2016 | Larceny              | SUM_LRC    

In [64]:
abeline_years = abeline.pivot('offense_name', 'year')
abeline_years.print_table(max_columns=None, max_column_width=30)

| offense_name                   | 2016 | 2015 | 2014 | 2013 | 2012 | 2011 | 2010 |
| ------------------------------ | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| Robbery                        |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Assault                        |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Larceny                        |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Aggravated Assault             |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Arson                          |    1 |    1 |    4 |    4 |    4 |    4 |    4 |
| Manslaughter by Negligence     |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Burglary                       |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Rape                           |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Motor Vehicle Theft            |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
| Murder and Nonnegligent Hom... |    1 |    1 |    1 |    1 |    1 |    1 |

In [65]:
abeline_sums = abeline.pivot('offense_name', aggregation=agate.Sum('reported'))
abeline_sums.print_table()

| offense_name         |    Sum |
| -------------------- | ------ |
| Robbery              |    893 |
| Assault              | 20,637 |
| Larceny              | 24,168 |
| Aggravated Assault   |  2,425 |
| Arson                |    525 |
| Manslaughter by N... |      2 |
| Burglary             |  7,766 |
| Rape                 |    442 |
| Motor Vehicle Theft  |  1,853 |
| Murder and Nonneg... |     36 |
